In [4]:
import autogen
import config

In [2]:
# Import necessary libraries
import imaplib
import email
from email.header import decode_header
import re

# Define a function to fetch the latest email details from a specific sender
def fetch_latest_email_details(user, password, sender):
    # Connect to the Gmail IMAP server securely using SSL
    mail = imaplib.IMAP4_SSL("imap.gmail.com")
    # Log in to the specified email account
    mail.login(user, password)
    # Select the inbox for operations
    mail.select("inbox")

    # Search for emails from the specified sender, 'FROM' filter used in IMAP search command
    status, messages = mail.search(None, f'(FROM "{sender}")')
    # Check if the search was successful
    if status != "OK":
        print("No emails found!")
        return None, None

    # Extract the ID of the latest email from the specified sender
    latest_email_id = messages[0].split()[-1]
    # Fetch the entire email (using RFC822 protocol) corresponding to the latest ID
    status, data = mail.fetch(latest_email_id, "(RFC822)")
    if status != "OK":
        print("Failed to fetch the email.")
        return None, None

    # Parse the email content into a message object
    msg = email.message_from_bytes(data[0][1])
    # Decode the email subject, if it's encoded
    subject = decode_header(msg["subject"])[0][0]
    if isinstance(subject, bytes):
        # Decode to string if subject is in bytes
        subject = subject.decode()

    # Initialize body content variable
    body = ""
    # Check if the email is multipart (has multiple parts like body, attachments)
    if msg.is_multipart():
        # Iterate through each part of the email
        for part in msg.walk():
            # Get the content type of the part
            ctype = part.get_content_type()
            # Check if the part is plain text or HTML content
            if ctype == "text/plain" or ctype == "text/html":
                # Decode and assign the content to 'body' variable
                body = part.get_payload(decode=True).decode()
                break
    else:
        # For non-multipart emails, directly decode and assign content
        body = msg.get_payload(decode=True).decode()

    # Define regex patterns to find the quoted text that indicates a reply or forward
    patterns = [
        r"On\s+\w{3}\s+\d{1,2}\.\s+\w{3}\s+\d{4}\s+at\s+\d{1,2}:\d{2},\s+<[^>]+>\s+wrote:",  # Pattern 1
        r"On\s+\w{3},\s+\d{1,2}\s+\w{3}\s+\d{4},\s+\d{1,2}:\d{2}\s*,\s*<[^>]+>\s+wrote:",  # Pattern 2
        r"On\s+\w{3},\s+\d{1,2}\s+\w{3}\s+\d{4}\s+at\s+\d{1,2}:\d{2}\s+[AP]M,\s+<[^>]+>\s+wrote:"  # Pattern 3
    ]

    # Initialize a variable to track the first match position of the patterns
    first_match_pos = len(body)
    # Iterate through each pattern to find a match in the email body
    for pattern in patterns:
        match = re.search(pattern, body, re.MULTILINE | re.IGNORECASE | re.DOTALL)
        # If a match is found and its position is before any previous match, update 'first_match_pos'
        if match and match.start() < first_match_pos:
            first_match_pos = match.start()

    # Extract everything before the first match as the latest message, otherwise consider the entire body
    latest_message = body[:first_match_pos].strip() if first_match_pos != len(body) else body

    # Return the decoded subject and the latest message content
    return subject, latest_message

In [5]:
# Set Gmail credentials and sender's email address
user = config.user_email
password = config.app_password
sender_email = config.sender

# Fetch latest email details (subject and body) from specified sender
subject, latest_email_body = fetch_latest_email_details(user, password, sender_email)

# Define file path for saving email content
file_path = "latest_email.txt"

# Save email content to file if fetched successfully
if subject and latest_email_body:
    # Write email subject and body to file if retrieval was successful
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(f"Subject: {subject}\n\n{latest_email_body}")
    print(f"Email saved to {file_path}")
else:
    # Print failure message if email fetching fails
    print("Failed to fetch email or no emails from the specified sender.")

Email saved to latest_email.txt
